In [76]:
# computation example taken from vitalik's amazing article:
def Evaluate(x):
    return x^3 + x + 5

# total constrains =
def EvaluateFlatten(x):
    v1 = x * x
    v2 = v1 * x
    out = v2 + x + 5
    return out

print(Evaluate(3))
print(EvaluateFlatten(3))

35
35


In [77]:
load("bls12-381.sage")
bls12381 = BLS12381()
e = Pairing(bls12381)

NumGates = 4
NumConstraints = 6

F1 = bls12381.F1
L = Matrix(F1, [
	[0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
	[5, 0, 0, 0, 0, 1]]
)
R = Matrix(F1, [
	[0, 1, 0, 0, 0, 0],
	[0, 1, 0, 0, 0, 0],
	[1, 0, 0, 0, 0, 0],
	[1, 0, 0, 0, 0, 0]]
)
O = Matrix(F1, [
	[0, 0, 0, 1, 0, 0],
	[0, 0, 0, 0, 1, 0],
	[0, 0, 0, 0, 0, 1],
	[0, 0, 1, 0, 0, 0]]
)

In [78]:
R1.<x> = PolynomialRing(F1)

M = [L, R, O]

PolyM = []

for m in M:
	PolyList = []
	for i in range(m.ncols()):
		points = []
		for j in range(m.nrows()):
			points.append([j+1, m[j, i]])

		Poly = R1.lagrange_polynomial(points).coefficients(sparse=False)

		if len(Poly) < m.nrows():
			diff = m.nrows() - len(Poly)
			for j in range(diff):
				Poly.append(0)
		# print("poly", Poly)
		PolyList.append(Poly)

	PolyM.append(Matrix(F1, PolyList)) # didn't quite understand this

In [79]:
S = vector(F1, [1, 3, 35, 9, 27, 30]) # solution vector

Lx = R1(list(S*PolyM[0]))
Rx = R1(list(S*PolyM[1]))
Ox = R1(list(S*PolyM[2]))

print("Lx:", Lx)
print("Rx:", Rx)
print("Ox:", Ox)

HxTx = Lx*Rx - Ox
Tx = R1((x-1)*(x-2)*(x-3)*(x-4))

print("HxTx:", HxTx)
print("Tx:", Tx)

Lx: 667068259203611232236298304289317359426147136656501314222009689354005275081806310740447938188169277339649045426626*x^3 + 2001204777610833696708894912867952078278441409969503942666029068062015825245418932221343814564507832018947136279932*x^2 + 1334136518407222464472596608578634718852294273313002628444019378708010550163612621480895876376338554679298090853189*x + 43
Rx: 1334136518407222464472596608578634718852294273313002628444019378708010550163612621480895876376338554679298090853263*x^3 + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559782*x^2 + 2668273036814444928945193217157269437704588546626005256888038757416021100327225242961791752752677109358596181706535*x + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559784
Ox: 667068259203611232236298304289317359426147136656501314222009689354005275081806310740447938188169277339649045426634*x^3 + 200120477761083369670

### Trusted setup phase

- generate random tau
- powers of tau with $G_1$ and $G_2$
- powers of tau with $t(\tau)$

In [86]:
def TrustedSetup(n, tx):
	tau = bls12381.F1.random_element()
	tau = bls12381.F1(2)

	print("tx:", tx)
	print("tau:", tau)

	tTau = tx(x=tau)
	tauG1, tauG2, tTauG1 = [], [], []
	for i in range(0, n):
		tauPow = tau^i
		tauG1.append(tauPow * e.G1)
		tauG2.append(tauPow * e.G2)
		tTauG1.append(tauPow * tTau * e.G1)
	return (tauG1, tauG2, tTauG1)

### Prover's steps

- calculate $h(x)$
- calculate $h(\tau)t(\tau)$
- calculate $[A]_1, [B]_2, [C]_1$

In [94]:
def hx(lx, rx, ox, tx):
	hxPoly = (lx*rx - ox).quo_rem(tx)
	assert(hxPoly[1] == 0)
	return hxPoly[0]

Hx = hx(Lx, Rx, Ox, Tx)

# print("Hx:", Hx)
# print("Hx[0]:", Hx[0])

TauG1, TauG2, TTauG1 = TrustedSetup(NumGates, Tx)
tau = bls12381.F1(2)
def evaluatePoly(poly, powersOfTau):
	deg = poly.degree()

	res = 0
	for i in range(deg):
		# print("i:", i, "poly[i]:", poly[i], "tTauG1:", tTauG1[i])
		res += poly[i] * powersOfTau[i]

	return res

A1 = 0
for i in range(NumConstraints):
	print("PolyM[0][i]:", i, PolyM[0][i])
	A1 += S[i] * evaluatePoly(PolyM[0][i], TauG1)

B2 = 0
for i in range(NumConstraints):
	B2 += S[i] * evaluatePoly(PolyM[1][i], TauG2)

# print("htauttau:", evaluatePoly(Hx, TTauG1))
# htautTau = HxTx(x=tau)

# res = 0
# for i in range(HxTx.degree()):
# 	taui = tau^i
# 	res += htautTau * taui * e.G1

HTauTTau = evaluatePoly(Hx, TTauG1)

# assert(HTauTTau == res)

C1 = HTauTTau
for i in range(NumConstraints):
	C1 += S[i] * evaluatePoly(PolyM[2][i], TauG1)

print(A1, B2, C1)

print(e.pair(A1, B2))
print(e.pair(C1, e.G2))

tx: x^4 + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559777*x^3 + 35*x^2 + 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559737*x + 24
tau: 2
PolyM[0][i]: 0 (4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559782, 3335341296018056161181491521446586797130735683282506571110048446770026375409031553702239690940846386698245227133165, 4002409555221667393417789825735904156556882819939007885332058136124031650490837864442687629129015664037894272559782, 667068259203611232236298304289317359426147136656501314222009689354005275081806310740447938188169277339649045426632)
PolyM[0][i]: 1 (8, 1334136518407222464472596608578634718852294273313002628444019378708010550163612621480895876376338554679298090853251, 5, 26682730368144449289451932171572694377045885466260052568880387574160211003272252429617917527526771093585961

### Verifier's steps

- verify $pairing([A]_1, [B]_2) = pairing([C]_1, G_2)$

In [95]:
# verify trusted setup
for i in range(len(TauG1)):
	assert(e.pair(TauG1[i], e.G2) == (e.pair(e.G1, TauG2[i])))

assert A1 in e.E1
assert A1 != e.E1(0)

assert B2 in e.E2
assert B2 != e.E2(0)

assert C1 in e.E1
assert C1 != e.E1(0)

assert(e.pair(A1, B2) == e.pair(C1, e.G2))

AssertionError: 

### Full Groth16 ahead

We need to add other variables like $\alpha, \beta, \gamma, \delta, r, s$ to make the protocol complete and sound as currently prover can easily invent values for $A, B, C$, and verifier has no way of verifying, if prover is malicious.

In [ ]:
# def CompleteTrustedSetup():